In [1]:
import tensorflow as tf
import numpy as np
import tensorflow.keras as keras
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import cifar10
import pandas as pd
import os

In [2]:
#数据加载
(image_train, label_train),\
(image_test, label_test) = cifar10.load_data()

In [3]:
#数据归一化
image_train_normalize = image_train / 255
image_test_normalize = image_test / 255

In [4]:
label_train_onehot = keras.utils.to_categorical(label_train)
label_test_onehot = keras.utils.to_categorical(label_test)

In [5]:
# Create Model
def identity_block(x_input, filters, kernel_size=(3,3)):
    filter1, filter2, filter3 = filters
    
    x_output = keras.layers.BatchNormalization(axis=3)(x_input)
    x_output = keras.layers.Activation('relu')(x_output)
    x_output = keras.layers.Conv2D(filters = filter1, kernel_size = (1,1), kernel_initializer='he_normal')(x_output)
    
    x_output = keras.layers.BatchNormalization(axis=3)(x_output)
    x_output = keras.layers.Activation('relu')(x_output)
    x_output = keras.layers.Conv2D(filters = filter2, kernel_size = kernel_size, padding='same', kernel_initializer='he_normal')(x_output)
    
    x_output = keras.layers.BatchNormalization(axis=3)(x_output)
    x_output = keras.layers.Activation('relu')(x_output)
    x_output = keras.layers.Conv2D(filters = filter3, kernel_size = (1,1), kernel_initializer='he_normal')(x_output)
    
    x_output = keras.layers.Add()([x_output, x_input])
    
    return x_output    

In [6]:
def convolutional_block(x_input, filters, kernel_size=(3,3), strides=(2,2), is_first=False):
    filter1, filter2, filter3 = filters
    
    if is_first:
        x_output = keras.layers.Conv2D(filters = filter1, kernel_size = (1,1),strides = strides, 
                                                  padding = 'same', kernel_initializer='he_normal')(x_input)
    else:
        x_output = keras.layers.BatchNormalization(axis=3)(x_input)
        x_output = keras.layers.Activation('relu')(x_output)
        x_output = keras.layers.Conv2D(filters = filter1, kernel_size = (1,1), strides = strides,
                                       padding = 'same', kernel_initializer='he_normal')(x_output)
    
    x_output = keras.layers.BatchNormalization(axis=3)(x_output)
    x_output = keras.layers.Activation('relu')(x_output)
    x_output = keras.layers.Conv2D(filters = filter2, kernel_size = kernel_size, padding='same', kernel_initializer='he_normal')(x_output)
    
    x_output = keras.layers.BatchNormalization(axis=3)(x_output)
    x_output = keras.layers.Activation('relu')(x_output)
    x_output = keras.layers.Conv2D(filters = filter3, kernel_size = (1,1), kernel_initializer='he_normal')(x_output)
    
    #断接通路卷积
    x_shortcut = keras.layers.Conv2D(filters = filter3, kernel_size = (1,1),
                                     strides = strides, kernel_initializer='he_normal')(x_input)
    x_shortcut = keras.layers.BatchNormalization(axis = 3)(x_shortcut)
    
    x_output = keras.layers.Add()([x_output, x_shortcut])
    
    return x_output

In [7]:
def build_resnet50(input_shape = (32,32,3), classes = 10):
    x_input = keras.layers.Input(input_shape)
    
    x_output = keras.layers.Conv2D(filters = 64, kernel_size = (7,7), padding = 'same', strides=(2,2),
                                   kernel_initializer = 'he_normal')(x_input)
    x_output = keras.layers.BatchNormalization(axis=3)(x_output)
    x_output = keras.layers.Activation('relu')(x_output)
    x_output = keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2), padding='same')(x_output)
    
    # ResNet Block
    x_output = convolutional_block(x_output, filters=[64, 64, 256], strides=(1,1), is_first=True)
    x_output = identity_block(x_output, filters = [64, 64, 256])
    x_output = identity_block(x_output, filters = [64, 64, 256])
    
    x_output = convolutional_block(x_output, filters=[128, 128, 512])
    x_output = identity_block(x_output, filters = [128, 128, 512])
    x_output = identity_block(x_output, filters = [128, 128, 512])
    x_output = identity_block(x_output, filters = [128, 128, 512])
    
    x_output = convolutional_block(x_output, filters=[256, 256, 1024])
    for _ in range(5):
        x_output = identity_block(x_output, filters = [256, 256, 1024])
        
    x_output = convolutional_block(x_output, filters=[512, 512, 2048])
    x_output = identity_block(x_output, filters = [512, 512, 2048])
    x_output = identity_block(x_output, filters = [512, 512, 2048])
    
    block_shape = keras.backend.int_shape(x_output)
    x_output = keras.layers.AveragePooling2D((block_shape[1], block_shape[2]))(x_output)
    
    x_output = keras.layers.Flatten()(x_output)
    x_output = keras.layers.Dense(1000, activation='relu')(x_output)
    x_output = keras.layers.Dense(1000, activation='relu')(x_output)
    x_output = keras.layers.Dense(classes, activation='softmax', kernel_initializer='he_normal')(x_output)
    
    model = keras.models.Model(inputs = x_input, outputs = x_output, name = 'ResNet50')
    return model

In [8]:
model = build_resnet50()
print(model.summary())

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 16, 16, 64)   9472        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 16, 16, 64)   256         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 16, 16, 64)   0           batch_normalization[0][0]        
___________________________________________________________________________________________

In [9]:
# Compile model
# sparse_categorical_crossentropy ?
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam',
              metrics = ['categorical_accuracy'])

In [10]:
# Train
imgGenerator = keras.preprocessing.image.ImageDataGenerator(
                        rotation_range=5,
                        shear_range=1,
                        width_shift_range=0.2,
                        height_shift_range=0.2,
                        horizontal_flip=True,
                        vertical_flip=False,
                        zoom_range=0.2,
                        fill_mode='nearest')

gen = imgGenerator.flow(image_train_normalize, label_train_onehot, batch_size=128)
history = model.fit(x=gen,validation_data=(image_test_normalize, label_test_onehot), epochs=10)

  ...
    to  
  ['...']
Train for 391 steps, validate on 10000 samples
Epoch 1/10
391/391 [==============================] - 46s 117ms/step - loss: 1.8442 - categorical_accuracy: 0.3595 - val_loss: 1.4897 - val_categorical_accuracy: 0.4518
Epoch 2/10
391/391 [==============================] - 30s 76ms/step - loss: 1.5487 - categorical_accuracy: 0.4358 - val_loss: 1.5882 - val_categorical_accuracy: 0.4440
Epoch 3/10
391/391 [==============================] - 30s 76ms/step - loss: 1.4426 - categorical_accuracy: 0.4815 - val_loss: 2.1973 - val_categorical_accuracy: 0.3242
Epoch 4/10
391/391 [==============================] - 30s 76ms/step - loss: 1.3563 - categorical_accuracy: 0.5148 - val_loss: 2.0659 - val_categorical_accuracy: 0.3194
Epoch 5/10
391/391 [==============================] - 30s 76ms/step - loss: 1.2984 - categorical_accuracy: 0.5364 - val_loss: 1.5014 - val_categorical_accuracy: 0.5178
Epoch 6/10
391/391 [==============================] - 30s 76ms/step - loss: 1.2228 - ca